In [10]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [11]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [12]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

True

In [13]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'reg_alpha': {
        'values': [0] + np.logspace(-5, -2, num=7).tolist()
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.00025118864
        },
    'batch_size': {
          'value': 64
        },
    'kernel_size': {
        'value': (3, 3)
        },
    'dropout': {
          'value': True
        },
    'pooling': {
          'value': 'max'
        },
    'batchnorm': {
          'value': True
        },
    'a_layers': {
          'value': 16
        },
    })

In [14]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'a_layers': {'values': [16, 19]},
                'batch_size': {'value': 64},
                'batchnorm': {'values': [True, False]},
                'dropout': {'values': [True, False]},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'kernel_size': {'values': [(3, 3), (5, 5)]},
                'learning_rate': {'value': 0.00025118864},
                'pooling': {'values': ['average', 'max']}}}


In [15]:
# sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: emrr8673
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/emrr8673


In [16]:
import tensorflow as tf

def create_model(kernel_size, dropout, pooling, batchnorm, n_layers, reg_alpha):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(64, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(128, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(256, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    if pooling == 'max':
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    else:
        model.add(tf.keras.layers.AveragePooling2D((2, 2)))

    if n_layers == 19:
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        model.add(tf.keras.layers.Conv2D(512, kernel_size, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
        if batchnorm:
            model.add(tf.keras.layers.BatchNormalization())


    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units=4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_alpha)))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model


In [17]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model(config["kernel_size"], config["dropout"], config["pooling"], config["batchnorm"], config["a_layers"], config["reg_alpha"])
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [18]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: c9m4ux2r with config:
wandb: 	a_layers: 16
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: average
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 28s 34ms/step - loss: 1.3959 - accuracy: 0.4862 - top@3_accuracy: 0.8092 - val_loss: 2.1002 - val_accuracy: 0.3950 - val_top@3_accuracy: 0.7718
Epoch 2/100
704/704 [==============================] - 23s 33ms/step - loss: 0.9441 - accuracy: 0.6688 - top@3_accuracy: 0.9069 - val_loss: 2.1470 - val_accuracy: 0.4298 - val_top@3_accuracy: 0.7580
Epoch 3/100
704/704 [==============================] - 24s 34ms/step - loss: 0.7269 - accuracy: 0.7503 - top@3_accuracy: 0.9380 - val_loss: 0.9178 - val_accuracy: 0.6998 - val_top@3_accuracy: 0.9154
Epoch 4/100
704/704 [==============================] - 21s 30ms/step - loss: 0.5867 - accuracy: 0.8028 - top@3_accuracy: 0.9559 - val_loss: 0.9223 - val_accuracy: 0.6994 - val_top@3_accuracy: 0.9026
Epoch 5/100
704/704 [==============================] - 21s 30ms/step - loss: 0.4732 - accuracy: 0.8403 - top@3_accuracy: 0.9681 - val_loss: 0.7557 - val_accuracy: 0.7644 - val_top@3_accuracy: 0.9376
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▇▇▇█████████
epoch/epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▆▆▇▇▇███████████
epoch/val_accuracy,▁▂▆▆▇▆▇████▇██▇███
epoch/val_loss,██▂▂▂▄▂▁▁▂▂▃▂▃▃▂▂▃
epoch/val_top@3_accuracy,▁▁▇▆▇▆▇████▇█▇▆▇█▇
test_acc,▁


wandb: Agent Starting Run: 8tsxgvfl with config:
wandb: 	a_layers: 16
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [3, 3]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: max
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 23s 31ms/step - loss: 1.6582 - accuracy: 0.3924 - top@3_accuracy: 0.7411 - val_loss: 1.3330 - val_accuracy: 0.5024 - val_top@3_accuracy: 0.8660
Epoch 2/100
704/704 [==============================] - 22s 31ms/step - loss: 1.0938 - accuracy: 0.6125 - top@3_accuracy: 0.8808 - val_loss: 1.1668 - val_accuracy: 0.6134 - val_top@3_accuracy: 0.8350
Epoch 3/100
704/704 [==============================] - 21s 30ms/step - loss: 0.8364 - accuracy: 0.7161 - top@3_accuracy: 0.9232 - val_loss: 0.7690 - val_accuracy: 0.7410 - val_top@3_accuracy: 0.9336
Epoch 4/100
704/704 [==============================] - 21s 30ms/step - loss: 0.6649 - accuracy: 0.7780 - top@3_accuracy: 0.9452 - val_loss: 1.0450 - val_accuracy: 0.6722 - val_top@3_accuracy: 0.8958
Epoch 5/100
704/704 [==============================] - 21s 30ms/step - loss: 0.5535 - accuracy: 0.8188 - top@3_accuracy: 0.9592 - val_loss: 0.8324 - val_accuracy: 0.7358 - val_top@3_accuracy: 0.9258
Epoch

...

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▄▆▄▁▆▇▆▇█▅▆▅
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▂▂▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▄▄█▅█▆▇▇▇▆
epoch/val_accuracy,▁█▁▁▁▆▁▁▁▁▆▁
epoch/val_loss,▂▁▃▅▅▆▇▇████
epoch/val_top@3_accuracy,▄█▄▄▁▁▄▄▁▁▄▁
test_acc,▁


wandb: Agent Starting Run: 7jov0z8o with config:
wandb: 	a_layers: 19
wandb: 	batch_size: 64
wandb: 	batchnorm: False
wandb: 	dropout: False
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: [5, 5]
wandb: 	learning_rate: 0.00025118864
wandb: 	pooling: max
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 54s 75ms/step - loss: 2.3027 - accuracy: 0.0992 - top@3_accuracy: 0.2942 - val_loss: 2.3026 - val_accuracy: 0.0958 - val_top@3_accuracy: 0.2894
Epoch 2/100
704/704 [==============================] - 53s 75ms/step - loss: 2.3026 - accuracy: 0.0992 - top@3_accuracy: 0.2946 - val_loss: 2.3027 - val_accuracy: 0.0958 - val_top@3_accuracy: 0.2878
Epoch 3/100
704/704 [==============================] - 53s 75ms/step - loss: 2.3026 - accuracy: 0.0973 - top@3_accuracy: 0.2973 - val_loss: 2.3027 - val_accuracy: 0.0950 - val_top@3_accuracy: 0.2878
Epoch 4/100
704/704 [==============================] - 53s 75ms/step - loss: 2.3026 - accuracy: 0.0981 - top@3_accuracy: 0.2984 - val_loss: 2.3028 - val_accuracy: 0.0950 - val_top@3_accuracy: 0.2896
Epoch 5/100
704/704 [==============================] - 53s 75ms/step - loss: 2.3026 - accuracy: 0.0991 - top@3_accuracy: 0.2963 - val_loss: 2.3028 - val_accuracy: 0.0950 - val_top@3_accuracy: 0.2896
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,██▂▄█▂█▄▇▅▁
epoch/epoch,▁▂▂▃▄▅▅▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▂▁▂▁▁▁
epoch/top@3_accuracy,▁▂▆▇▄▇█▇▇▆▅
epoch/val_accuracy,██▁▁▁▁▁▁▁▁▁
epoch/val_loss,▁▄▅▇▆▆▆▇▇▇█
epoch/val_top@3_accuracy,▇▁▁██▃▁▃▃▁▁
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
